# Content-based Recommender 

### Another Example: scikit-learn: TF/IDF and cosine similarity for computer science papers
[Computer Science Papers](http://www.markhneedham.com/blog/2016/07/27/scitkit-learn-tfidf-and-cosine-similarity-for-computer-science-papers/)

Task:  Finding similar computer science papers to read based on the similarity between titles using TF-IDF

What is TF-IDF (Term Frequency - Inverse Document Frequency)

<img src="pic/tfidf.jpg">

We’ve got one file per paper which contains the title of the paper. We first need to iterate through that directory and build an array containing the papers:

In [91]:
import glob
 
corpus = []
for file in glob.glob("data/papers/*.txt"):
    with open(file, "r") as paper:
        corpus.append((file, paper.read()))

Check the titles of the first 10 papers. 

In [92]:
corpus[0:10]

[('data/papers/100000.txt', 'Inside risks: the clock grows at midnight'),
 ('data/papers/100226.txt', 'Coherent functions and program checkers'),
 ('data/papers/100228.txt', 'The wakeup problem'),
 ('data/papers/100231.txt', 'Efficient robust parallel computations'),
 ('data/papers/100262.txt',
  'An optimal algorithm for on-line bipartite matching'),
 ('data/papers/100269.txt',
  'One-way functions are necessary and sufficient for secure signatures'),
 ('data/papers/100270.txt',
  'Pseudo-random generators under uniform assumptions'),
 ('data/papers/100272.txt',
  'Witness indistinguishable and witness hiding protocols'),
 ('data/papers/100273.txt',
  'Public-key cryptosystems provably secure against chosen ciphertext attacks'),
 ('data/papers/100287.txt', 'The round complexity of secure protocols')]

Next we’ll build a TF/IDF matrix for each paper:

In [93]:
from sklearn.feature_extraction.text import TfidfVectorizer
 
tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform([content for file, content in corpus[:]])

Next we’ll write a function that will find us the top n similar papers based on cosine similarity:

In [94]:
from sklearn.metrics.pairwise import linear_kernel
 
def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

Let’s try it out:

In [95]:
corpus[1619]

('data/papers/221215.txt',
 'TOTEM: a reliable ordered delivery protocol for interconnected local-area networks')

In [96]:
for index, score in find_similar(tfidf_matrix, 1619):
       print(score, corpus[index])

0.917540397202 ('data/papers/852338.txt', 'A reliable ordered delivery protocol for interconnected local area networks')
0.248736845733 ('data/papers/800897.txt', 'Interconnection of broadband local area networks')
0.207309089025 ('data/papers/103726.txt', 'High-speed local area networks and their performance: a survey')
0.204166719869 ('data/papers/161736.txt', 'High-speed switch scheduling for local-area networks')
0.198514433132 ('data/papers/627363.txt', 'Algorithms for Distributed Query Processing in Broadcast Local Area Networks')


It’s pretty good for finding duplicate papers!

In [97]:
corpus[1599]

('data/papers/217470.txt',
 'A reliable multicast framework for light-weight sessions and application level framing')

In [98]:
for index, score in find_similar(tfidf_matrix, 1599):
       print (score, corpus[index])

1.0 ('data/papers/270863.txt', 'A reliable multicast framework for light-weight sessions and application level framing')
0.139643354066 ('data/papers/218325.txt', 'The KryptoKnight family of light-weight protocols for authentication and key distribution')
0.134763799612 ('data/papers/1251445.txt', 'ALMI: an application level multicast infrastructure')
0.117630311817 ('data/papers/125160.txt', 'Ordered and reliable multicast communication')
0.117630311817 ('data/papers/128741.txt', 'Ordered and reliable multicast communication')


But sometimes it identifies duplicates that aren’t identical:

In [99]:
corpus[5784]

('data/papers/RFC2616.txt', 'Hypertext Transfer Protocol -- HTTP/1.1')

In [100]:
for index, score in find_similar(tfidf_matrix, 5784):
       print (score, corpus[index])

1.0 ('data/papers/RFC1945.txt', 'Hypertext Transfer Protocol -- HTTP/1.0')
1.0 ('data/papers/RFC2068.txt', 'Hypertext Transfer Protocol -- HTTP/1.1')
0.232865694216 ('data/papers/131844.txt', 'XTP: the Xpress Transfer Protocol')
0.138876842331 ('data/papers/RFC1866.txt', 'Hypertext Markup Language - 2.0')
0.104775586915 ('data/papers/760249.txt', 'On the transfer of control between contexts')


Finally, let us creat a CSV file containing to 5 similar papers for each paper. 

In [108]:
import csv
with open("./output/similarities.csv", "w") as similarities_file:
    writer = csv.writer(similarities_file, delimiter = ",")

    for me_index, item in enumerate(corpus):
        similar_documents =  [(corpus[index], score) for index, score in find_similar(tfidf_matrix, me_index)]
        me = corpus[me_index]

        document_id = me[0].split("/")[2].split(".")[0]

        for ((raw_similar_document_id, title), score) in similar_documents:
            similar_document_id = raw_similar_document_id.split("/")[2].split(".")[0]
            writer.writerow([document_id, me[1], similar_document_id, title, score])

In [ ]:
Print 5 top similar papers for each paper as long as the similarity is greater than 0.5.

In [111]:
items = []

with open("./output/similarities.csv", "r") as similarities_file:
    reader = csv.reader(similarities_file, delimiter = ",")
    for row in reader:
        lst = list(row)
        lst[4] = float(lst[4])
        items.append(tuple(lst))

by_similarity = sorted(items, key = lambda x: x[4], reverse = True)

for similar_item in [item for item in by_similarity if 0.5 < item[4] < 0.99]:
    print (similar_item) 
    print ('\n')

('221215', 'TOTEM: a reliable ordered delivery protocol for interconnected local-area networks', '852338', 'A reliable ordered delivery protocol for interconnected local area networks', 0.917540397202)


('852338', 'A reliable ordered delivery protocol for interconnected local area networks', '221215', 'TOTEM: a reliable ordered delivery protocol for interconnected local-area networks', 0.917540397202)


('146935', 'The generalized tree quorum protocol: an efficient approach for managing replicated data', '671977', 'The Tree Quorum Protocol: An Efficient Approach for Managing Replicated Data', 0.914810942048)


('146935', 'The generalized tree quorum protocol: an efficient approach for managing replicated data', '94419', 'The tree quorum protocol: an efficient approach for managing replicated data', 0.914810942048)


('671977', 'The Tree Quorum Protocol: An Efficient Approach for Managing Replicated Data', '146935', 'The generalized tree quorum protocol: an efficient approach for manag

# Getting started with a quick-and-easy k-nearest neighbor classifier

THis is a tinny example to show the idea of KNN for recommender development. 

In [112]:
import numpy as np
from sklearn.neighbors import NearestNeighbors

# First let's create a dataset called X, with 6 records and 2 features each.
X = np.array([[-1, 2], [4, -4], [-2, 1], [-1, 3], [-3, 2], [-1, 4]])

# Next we will instantiate a nearest neighbor object, and call it nbrs. Then we will fit it to dataset X.
nbrs = NearestNeighbors(n_neighbors=3, algorithm='ball_tree').fit(X)

# Let's find the k-neighbors of each point in object X. To do that we call the kneighbors() function on object X.
distances, indices = nbrs.kneighbors(X)

# Let's print out the indices of neighbors for each record in object X.
indices

array([[0, 3, 2],
       [1, 2, 0],
       [2, 4, 0],
       [3, 5, 0],
       [4, 2, 0],
       [5, 3, 0]])

In [113]:
distances

array([[ 0.        ,  1.        ,  1.41421356],
       [ 0.        ,  7.81024968,  7.81024968],
       [ 0.        ,  1.41421356,  1.41421356],
       [ 0.        ,  1.        ,  1.        ],
       [ 0.        ,  1.41421356,  2.        ],
       [ 0.        ,  1.        ,  2.        ]])

Imagine you have a new incoming data point. It contains the values -2 and 4. To search object X and identify the most 
similar record, all you need to do is call the kneighbors() function on the new incoming data p

In [114]:
print(nbrs.kneighbors([[-2, 4]])) 

(array([[ 1.        ,  1.41421356,  2.23606798]]), array([[5, 3, 0]]))


The results indicate that the record that has neighbors with the indices [5, 3, 0] is the most similar to the new incoming 
data point. If you look back at the records in X, that is the last record: [-1, 4]. Just based on a quick glance you 
can see that, indeed, the last record in object X is the one that is most similar to this new incoming data point [-2, 4]. 